In [1]:
from rag_module.rag import RAG
import os

os.environ["embedding_path"] = "./embeddings/guidance_framework_2/"

c:\Users\SjoerdO\AppData\Local\pypoetry\Cache\virtualenvs\rag--7y4UdRY-py3.13\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rag = RAG(scope_model_id='73e8819c680c49b2a511a5efbe04876b')

Loading default flashrank model for language en
Default Model: ms-marco-MiniLM-L-12-v2
Loading FlashRankRanker model ms-marco-MiniLM-L-12-v2 (this message can be suppressed by setting verbose=0)
Loading model FlashRank model ms-marco-MiniLM-L-12-v2...
Successfully loaded model from run 73e8819c680c49b2a511a5efbe04876b
Loaded features: ['top_sparse_score', 'top_dense_score']


In [4]:
# load all dfs in eval_data/SyntheticQA/QA_sets/
import os
import pandas as pd

# Get all CSV files in the directory
directory = "eval_data/SyntheticQA/QA_sets/"
csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

# Loop through the files and read them into DataFrames
dataframes = []
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concatenate all DataFrames into one
df_gf = pd.concat(dataframes, ignore_index=True)
df_gf = df.groupby('chunk_id').first()

print(f'n queries guidance framework: {len(df_gf)}')

n queries guidance framework: 55


## User tests

In [6]:
queries = [
    'What is the main reason for avoiding the implementation of complex business logic within Power BI?',
    'What is the purpose of the sprint retrospective in the Scrum process?',
    'What are some potential drawbacks of using Lombok in Java development?',
    'Which database should I use for graphs?',
    'what is an ETL pipeline?',
    'List three use cases for AI in the financial sector.',
    'What information should I include in a git commit message?',
    'Which plot should I use to visualize six distributions on the same scale?'
]

In [7]:
responses = [rag.query(query) for query in queries]

Batches: 100%|██████████| 1/1 [00:00<00:00, 52.77it/s]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Batches: 100%|██████████| 1/1 [00:00<00:00, 43.14it/s]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Batches: 100%|██████████| 1/1 [00:00<00:00, 38.43it/s]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 20.000000 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Batches: 100%|██████████| 1/1 [00:00<00:00, 35.59it/s]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 20.000000 seconds
INFO:httpx:HTTP Request: POST https:

In [8]:
# create a dataframe from responses
import pandas as pd
df = pd.DataFrame(responses)

In [9]:
df

,chunks,query,scope_prediction,scope_probabilities,answer,counterfactual
0,"[{'filename': '01-power-bi.md', 'text': '# Pow...",What is the main reason for avoiding the imple...,0,"[0.6818717116033882, 0.27442004770119843, 0.04...",The main reason for avoiding the implementatio...,NaN
1,"[{'filename': 'retrospectives.md', 'text': '# ...",What is the purpose of the sprint retrospectiv...,0,"[0.9101098278920184, 0.08663759528188793, 0.00...",The purpose of the Sprint Retrospective in the...,NaN
2,[{'filename': '01-code-generation-libraries.md...,What are some potential drawbacks of using Lom...,1,"[0.21397526214846513, 0.6817708980864439, 0.10...",Some potential drawbacks of using Lombok in Ja...,What are some advantages of using Lombok in Ja...
3,"[{'filename': 'azure-managed-databases.md', 't...",Which database should I use for graphs?,1,"[0.036034671511195875, 0.5314615939241095, 0.4...",You should use **Azure Cosmos DB with the Grem...,What is the recommended managed offering for g...
4,[{'filename': 'ADR-012 Logging Strategy for ET...,what is an ETL pipeline?,1,"[0.03188229001381606, 0.5250100303578046, 0.44...",An ETL pipeline is a data processing framework...,How can we effectively monitor the execution o...
5,"[{'filename': 'ethics-data.md', 'text': '# Eth...",List three use cases for AI in the financial s...,2,"[0.033860078641397146, 0.40228940735587326, 0....",Here are three use cases for AI in the financi...,NaN
6,"[{'filename': 'index.md', 'text': '### Structu...",What information should I include in a git com...,2,"[0.027917661007111204, 0.35540117165723134, 0....",A good Git commit message should include the f...,NaN
7,[{'filename': '60-data-analysis-and-feature-en...,Which plot should I use to visualize six distr...,2,"[0.016292340500934595, 0.2771850837680961, 0.7...",To visualize six distributions on the same sca...,NaN


In [ ]:
# export the dataframe to a csv file
df.to_csv('rag_usertest_responses.csv', index=False)